# Linear Discriminant Analysis

### Objective

Training a Linear Discriminant Analysis(LDA) model to check if the product has been shipped or canceled.

### Problem Statement

XYZ.com is an e-commerce company based in Argentina. Due to the covid crisis and lockdown XYZ.com is facing lots of issues from the dealer and the shipment team.  XYZ.com has lots of product data where various shipping and sales details of each product have been mentioned. XYZ.com wants to find out which of the products has been shipped and which of the products has been canceled to reduce customer escalation. As a data-scientist, We have to train an LDA(Linear Discriminant Analysis) model to predict which of the product has been shipped and which of the product has been canceled.

### 1. Import necessary libraries.

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix,roc_curve,accuracy_score,auc,roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

### 2. Display a sample of five rows of the data frame

In [5]:
df = pd.read_csv('/content/drive/MyDrive/sales_data_sample.csv',encoding='unicode_escape')

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
df.sample(n=5)

,ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,MONTH_ID,YEAR_ID,...,ADDRESSLINE1,ADDRESSLINE2,CITY,STATE,POSTALCODE,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE
920,10244,24,58.09,3,1394.16,4/29/2004 0:00,Shipped,2,4,2004,...,"C/ Moralzarzal, 86",NaN,Madrid,NaN,28034,Spain,EMEA,Freyre,Diego,Small
1964,10350,25,60.34,10,1508.50,12/2/2004 0:00,Shipped,4,12,2004,...,"C/ Moralzarzal, 86",NaN,Madrid,NaN,28034,Spain,EMEA,Freyre,Diego,Small
1649,10194,49,100.00,1,5760.93,11/25/2003 0:00,Shipped,4,11,2003,...,"2, rue du Commerce",NaN,Lyon,NaN,69004,France,EMEA,Saveley,Mary,Medium
2539,10167,29,100.00,5,2940.02,10/23/2003 0:00,Cancelled,4,10,2003,...,?kergatan 24,NaN,Boras,NaN,S-844 67,Sweden,EMEA,Larsson,Maria,Small
1675,10190,42,76.19,3,3199.98,11/19/2003 0:00,Shipped,4,11,2003,...,"C/ Moralzarzal, 86",NaN,Madrid,NaN,28034,Spain,EMEA,Freyre,Diego,Medium


### 3. Check the shape of the data (number of rows and column). Check the general information about the dataframe using .info() method.

In [7]:
def basic_info(df):
    '''checking basic information & shape about the dataframe'''
    temp=df.copy(deep=True)
    print("Shape of the dataset",df.shape)
    print("*"*30)
    print(df.info())

basic_info(df)

Shape of the dataset (2823, 25)
******************************
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2823 entries, 0 to 2822
Data columns (total 25 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ORDERNUMBER       2823 non-null   int64  
 1   QUANTITYORDERED   2823 non-null   int64  
 2   PRICEEACH         2823 non-null   float64
 3   ORDERLINENUMBER   2823 non-null   int64  
 4   SALES             2823 non-null   float64
 5   ORDERDATE         2823 non-null   object 
 6   STATUS            2823 non-null   object 
 7   QTR_ID            2823 non-null   int64  
 8   MONTH_ID          2823 non-null   int64  
 9   YEAR_ID           2823 non-null   int64  
 10  PRODUCTLINE       2823 non-null   object 
 11  MSRP              2823 non-null   int64  
 12  PRODUCTCODE       2823 non-null   object 
 13  CUSTOMERNAME      2823 non-null   object 
 14  PHONE             2823 non-null   object 
 15  ADDRESSLINE1      2823 non

### 4.Check the percentage of missing values in each column of the data frame.

In [8]:
def check_missing_values(df):
    '''Function to check the missing data percentage'''
    print("percentage of missing data of dataframe \n",df.isnull().sum()/len(df)*100)

check_missing_values(df)

percentage of missing data of dataframe 
 ORDERNUMBER          0.000000
QUANTITYORDERED      0.000000
PRICEEACH            0.000000
ORDERLINENUMBER      0.000000
SALES                0.000000
ORDERDATE            0.000000
STATUS               0.000000
QTR_ID               0.000000
MONTH_ID             0.000000
YEAR_ID              0.000000
PRODUCTLINE          0.000000
MSRP                 0.000000
PRODUCTCODE          0.000000
CUSTOMERNAME         0.000000
PHONE                0.000000
ADDRESSLINE1         0.000000
ADDRESSLINE2        89.302161
CITY                 0.000000
STATE               52.639036
POSTALCODE           2.692171
COUNTRY              0.000000
TERRITORY           38.044633
CONTACTLASTNAME      0.000000
CONTACTFIRSTNAME     0.000000
DEALSIZE             0.000000
dtype: float64


- Here we can see the column ADDRESSLINE2 has  89.3% of missing values, STATE has 52.6% of missing values.
- POSTALCODE has around 2.69% and TERRITORY has  around 38%.

- We will drop the ADDRESSLINE2 column and impute other columns with most occured values with the respective column.

### 5. Check if there are any duplicate rows.

In [9]:
def check_duplicate(df):
    duplicate=df[df.duplicated()]
    return duplicate

duplicate=check_duplicate(df)
duplicate

,ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,MONTH_ID,YEAR_ID,...,ADDRESSLINE1,ADDRESSLINE2,CITY,STATE,POSTALCODE,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE


- As we can see, There is no duplicate columns which is a good thing for us.

### 6. Write a function that will impute missing values of the columns “STATE”, “POSTALCODE”,“TERRITORY” with its most occurring label.  

In [10]:
def impute_most_occur(df):
    df=df.copy(deep=True)
    temp_state=df['STATE'].value_counts().index[0]
    df['STATE'].fillna(value=temp_state,inplace=True)
    temp_postal_code=df['POSTALCODE'].value_counts().index[0]
    df['POSTALCODE'].fillna(value=temp_postal_code,inplace=True)
    temp_territory=df['TERRITORY'].value_counts().index[0]
    df['TERRITORY'].fillna(value=temp_territory,inplace=True)
    return df

df=impute_most_occur(df)

In [11]:
check_missing_values(df)
print("*"*50)
basic_info(df)

percentage of missing data of dataframe 
 ORDERNUMBER          0.000000
QUANTITYORDERED      0.000000
PRICEEACH            0.000000
ORDERLINENUMBER      0.000000
SALES                0.000000
ORDERDATE            0.000000
STATUS               0.000000
QTR_ID               0.000000
MONTH_ID             0.000000
YEAR_ID              0.000000
PRODUCTLINE          0.000000
MSRP                 0.000000
PRODUCTCODE          0.000000
CUSTOMERNAME         0.000000
PHONE                0.000000
ADDRESSLINE1         0.000000
ADDRESSLINE2        89.302161
CITY                 0.000000
STATE                0.000000
POSTALCODE           0.000000
COUNTRY              0.000000
TERRITORY            0.000000
CONTACTLASTNAME      0.000000
CONTACTFIRSTNAME     0.000000
DEALSIZE             0.000000
dtype: float64
**************************************************
Shape of the dataset (2823, 25)
******************************
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2823 entries, 0 to 2822
Data 

### 7. Drop “ADDRESSLINE2”,”ORDERDATE”,”PHONE” column.

In [12]:
df.drop(['ADDRESSLINE2','ORDERDATE','PHONE'],axis=1,inplace=True)

In [13]:
check_missing_values(df)
print("*"*50)
basic_info(df)

percentage of missing data of dataframe 
 ORDERNUMBER         0.0
QUANTITYORDERED     0.0
PRICEEACH           0.0
ORDERLINENUMBER     0.0
SALES               0.0
STATUS              0.0
QTR_ID              0.0
MONTH_ID            0.0
YEAR_ID             0.0
PRODUCTLINE         0.0
MSRP                0.0
PRODUCTCODE         0.0
CUSTOMERNAME        0.0
ADDRESSLINE1        0.0
CITY                0.0
STATE               0.0
POSTALCODE          0.0
COUNTRY             0.0
TERRITORY           0.0
CONTACTLASTNAME     0.0
CONTACTFIRSTNAME    0.0
DEALSIZE            0.0
dtype: float64
**************************************************
Shape of the dataset (2823, 22)
******************************
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2823 entries, 0 to 2822
Data columns (total 22 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ORDERNUMBER       2823 non-null   int64  
 1   QUANTITYORDERED   2823 non-null   int64  
 2   P

### 8. Convert the labels of the STATUS column to 0 and 1. For Shipped assign value 1 and for all other labels (i.e. ‘Cancelled’,’ Resolved’,’ On Hold’,’ In Process’, 'Disputed') assign 0. Note we will consider everything apart from Shipped as cancel (i.e. 0).


In [14]:
df['STATUS']=df['STATUS'].map({'Shipped':1,'Cancelled':0,'Resolved':0,'On Hold':0,'In Process':0,'Disputed':0})

In [15]:
df['STATUS'].value_counts()

1    2617
0     206
Name: STATUS, dtype: int64

- Note: The target class is highly imbalanced.

### 9. Encode the categorical features using dummy encoding

In [16]:
df = pd.get_dummies(df,drop_first=True)

In [17]:
df.shape

(2823, 634)

- Note since we had so many categorical features, after encoding there are huge number of dimensions.

### 10. Separate the target and independent features

In [18]:
X = df.drop('STATUS',axis=1)
y = df['STATUS']

### 11. Split the dataset into two parts (i.e. 80% train and 20% test) using random_state=42.

In [19]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

print(X_train.shape,X_test.shape)
print(y_train.shape,y_test.shape)

(2258, 633) (565, 633)
(2258,) (565,)


### 11. Scale the data

In [20]:
## Scaling the data using MINMAXSCALAR
mm = MinMaxScaler()

X_train.iloc[:,:] = mm.fit_transform(X_train.iloc[:,:])
X_test.iloc[:,:] = mm.transform(X_test.iloc[:,:])

## LDA

### Traning a RandomForest Classfier model before applying LDA

In [21]:
rf = RandomForestClassifier(max_depth=3,n_estimators=25)
rf.fit(X_train,y_train)
y_train_pred = rf.predict(X_train)
y_test_pred = rf.predict(X_test)

print("Train Accuracy",accuracy_score(y_train,y_train_pred))
print("Test Accuracy",accuracy_score(y_test,y_test_pred))
print("*"*50)
print("Train confusion matrix",'\n',confusion_matrix(y_train,y_train_pred))
print("Test confusion matrix",'\n',confusion_matrix(y_test,y_test_pred))

Train Accuracy 0.9317980513728964
Test Accuracy 0.9079646017699115
**************************************************
Train confusion matrix 
 [[   0  154]
 [   0 2104]]
Test confusion matrix 
 [[  0  52]
 [  0 513]]


### Training a Linear Discriminant Analysis(LDA) model on the train data. Do fit_transform on the train data and only transform on the test data.

In [22]:
lda = LDA(n_components=1)

X_train = lda.fit_transform(X_train, y_train)
X_test = lda.transform(X_test)

In [23]:
X_train[:5],X_test[:5]

(array([[ 0.60496511],
        [ 0.28984435],
        [ 0.6028427 ],
        [-0.21719613],
        [ 0.27988057]]),
 array([[ 1.26394182],
        [ 2.52900911],
        [ 1.33478261],
        [-2.94060547],
        [ 0.7395123 ]]))

### Training a random-forest model on the transformed data  and print the accuracy of train and test data. Take max_depth=3 and n_estimators=25

In [24]:
rf=RandomForestClassifier(max_depth=3,n_estimators=25)
rf.fit(X_train,y_train)
y_train_pred=rf.predict(X_train)
y_test_pred=rf.predict(X_test)

print("Train Accuracy",accuracy_score(y_train,y_train_pred))
print("Test Accuracy",accuracy_score(y_test,y_test_pred))
print("*"*50)
print("Train confusion matrix",'\n',confusion_matrix(y_train,y_train_pred))
print("Test confusion matrix",'\n',confusion_matrix(y_test,y_test_pred))

Train Accuracy 0.9534986713906112
Test Accuracy 0.9221238938053097
**************************************************
Train confusion matrix 
 [[  93   61]
 [  44 2060]]
Test confusion matrix 
 [[ 22  30]
 [ 14 499]]


### Conclusion

- As we can see we True negative and False negative points is zero before applying LDA but after applying LDA we are getting some amount of True negative and false negative points.
- Due to class imbalance, the majority class has high recall but minority class has poor recall.


### Happy Learning:)